In [168]:
# DATA2001 Assignment

from IPython.display import HTML
HTML('''
    <style> body {font-family: "Roboto Condensed Light", "Roboto Condensed";} h2 {padding: 10px 12px; background-color: #E64626; position: static; color: #ffffff; font-size: 40px;} .text_cell_render p { font-size: 15px; } .text_cell_render h1 { font-size: 30px; } h1 {padding: 10px 12px; background-color: #E64626; color: #ffffff; font-size: 40px;} .text_cell_render h3 { padding: 10px 12px; background-color: #0148A4; position: static; color: #ffffff; font-size: 20px;} h4:before{ 
    content: "@"; font-family:"Wingdings"; font-style:regular; margin-right: 4px;} .text_cell_render h4 {padding: 8px; font-family: "Roboto Condensed Light"; position: static; font-style: italic; background-color: #FFB800; color: #ffffff; font-size: 18px; text-align: center; border-radius: 5px;}input[type=submit] {background-color: #E64626; border: solid; border-color: #734036; color: white; padding: 8px 16px; text-decoration: none; margin: 4px 2px; cursor: pointer; border-radius: 20px;}</style>
''')

# Assignment

In [3]:
from sqlalchemy import create_engine, text, types
import psycopg2
import psycopg2.extras
import json
import os
import pandas as pd
import geoalchemy2
import geopandas as gpd


credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [4]:
db, conn = pgconnect(credentials)
conn.execute(text("set search_path to public"))

Connected successfully.


### Loading Data into Python from .csv, .txt and .shp

In [5]:
data_path = "data"

population_df = pd.read_csv(f"{data_path}/Population.csv")
businesses_df = pd.read_csv(f"{data_path}/Businesses.csv")
income_df = pd.read_csv(f"{data_path}/sa2_income.csv")
postcode_df = pd.read_csv(f"{data_path}/sa2_postcode.csv")
stops_df = pd.read_csv(f"{data_path}/Stops.txt")

sa2_gdf = gpd.read_file(f"{data_path}/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")
primary_catchments = gpd.read_file(f"{data_path}/catchments/catchments_primary.shp")
secondary_catchments = gpd.read_file(f"{data_path}/catchments/catchments_secondary.shp")

### Data cleaning before ingestion into database

### Cleaning population

In [6]:
cleaned_population_df = population_df.copy()

cleaned_population_df.columns = (
    cleaned_population_df.columns
    .str.lower()
    .str.replace('-', '_')
    .str.replace(' ', '_')
)


### Cleaning Businesses

In [7]:
cleaned_businesses_df = businesses_df.copy()
for col in cleaned_businesses_df.select_dtypes(include="object"):
    cleaned_businesses_df[col] = cleaned_businesses_df[col].str.strip()
cleaned_businesses_df.rename(columns={
    "0_to_50k_businesses": "b_0_50k",
    "50k_to_200k_businesses": "b_50k_200k",
    "200k_to_2m_businesses": "b_200k_2m",
    "2m_to_5m_businesses": "b_2m_5m",
    "5m_to_10m_businesses": "b_5m_10m",
    "10m_or_more_businesses": "b_10m_plus",
    "total_businesses": "total"
}, inplace=True)


### Cleaning Income

In [8]:
cleaned_income_df = income_df.copy()

cleaned_income_df.rename(columns={
    "SA2_code": "sa2_code",
    "SA2_name": "sa2_name"
}, inplace=True)
cleaned_income_df['income_earners'].fillna(cleaned_income_df['income_earners'].median(), inplace=True)
cleaned_income_df['median_income'].fillna(cleaned_income_df['median_income'].median(), inplace=True)
cleaned_income_df['mean_income'].fillna(cleaned_income_df['mean_income'].median(), inplace=True)
cleaned_income_df['gini_coefficient'].fillna(cleaned_income_df['gini_coefficient'].median(), inplace=True)

/var/folders/7w/0bgjmph51g3fjv7yfs8y17xc0000gn/T/ipykernel_42220/4119000927.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_income_df['income_earners'].fillna(cleaned_income_df['income_earners'].median(), inplace=True)
/var/folders/7w/0bgjmph51g3fjv7yfs8y17xc0000gn/T/ipykernel_42220/4119000927.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the i

### Cleaning Postcode

In [9]:
cleaned_postcode_df = postcode_df.copy()

postcode_df.rename(columns={
    "SA2_code": "sa2_code",
    "SA2_name": "sa2_name"
}, inplace=True)

### Cleaning Stops

In [10]:
cleaned_stops_df = stops_df.copy()

cleaned_stops_df.columns = cleaned_stops_df.columns.str.strip().str.lower()

for col in cleaned_stops_df.select_dtypes(include='object'):
    cleaned_stops_df[col] = cleaned_stops_df[col].str.strip()

cleaned_stops_df.drop(columns=['parent_station', 'stop_code', 'platform_code', 'location_type'], inplace=True)


### Cleaning Geodata

In [11]:
def clean_gdf(gdf):
    gdf = gdf.copy()
    gdf.columns = gdf.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')

    for col in gdf.select_dtypes(include="object"):
        gdf[col] = gdf[col].str.strip()

    return gdf

cleaned_sa2_gdf = clean_gdf(sa2_gdf)
cleaned_primary_catchment= clean_gdf(primary_catchments)
cleaned_secondary_catchment = clean_gdf(secondary_catchments)

target_crs = "EPSG:4326"
cleaned_sa2_gdf = cleaned_sa2_gdf.to_crs(target_crs)
cleaned_primary_catchment = cleaned_primary_catchment.to_crs(target_crs)
cleaned_secondary_catchment = cleaned_secondary_catchment.to_crs(target_crs)

cleaned_school_catchment = pd.concat([cleaned_primary_catchment, cleaned_secondary_catchment], ignore_index=True)
cleaned_school_catchment.drop(columns=[
    "kindergart", "year1", "year2", "year3", "year4", "year5", "year6",
    "year7", "year8", "year9", "year10", "year11", "year12", "priority"
], inplace=True)
cleaned_school_catchment.rename(columns={
    "use_id": "school_id",
    "use_desc": "school_name",
    "geometry": "geom"  
}, inplace=True)
print(cleaned_school_catchment.info())


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2098 entries, 0 to 2097
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   school_id    2098 non-null   object  
 1   catch_type   2098 non-null   object  
 2   school_name  2098 non-null   object  
 3   add_date     1707 non-null   object  
 4   geom         2098 non-null   geometry
dtypes: geometry(1), object(4)
memory usage: 82.1+ KB
None


### Load in Geodata

### 1.2. Ingestion

We now need to ingest all of the collected data into the database

In [12]:
conn.execute(text("""
DROP TABLE IF EXISTS population CASCADE;
CREATE TABLE population (
    sa2_code BIGINT PRIMARY KEY,
    sa2_name TEXT,
    "0_4_people" INTEGER,
    "5_9_people" INTEGER,
    "10_14_people" INTEGER,
    "15_19_people" INTEGER,
    "20_24_people" INTEGER,
    "25_29_people" INTEGER,
    "30_34_people" INTEGER,
    "35_39_people" INTEGER,
    "40_44_people" INTEGER,
    "45_49_people" INTEGER,
    "50_54_people" INTEGER,
    "55_59_people" INTEGER,
    "60_64_people" INTEGER,
    "65_69_people" INTEGER,
    "70_74_people" INTEGER,
    "75_79_people" INTEGER,
    "80_84_people" INTEGER,
    "85_and_over_people" INTEGER,
    total_people INTEGER
);
                  
DROP TABLE IF EXISTS businesses CASCADE;
CREATE TABLE businesses (
    industry_code TEXT,
    industry_name TEXT,
    sa2_code BIGINT,
    sa2_name TEXT,
    b_0_50k INTEGER,
    b_50k_200k INTEGER,
    b_200k_2m INTEGER,
    b_2m_5m INTEGER,
    b_5m_10m INTEGER,
    b_10m_plus INTEGER,
    total INTEGER,
    PRIMARY KEY (sa2_code, industry_code)

);
                  
DROP TABLE IF EXISTS income CASCADE;
CREATE TABLE income (
    sa2_code BIGINT PRIMARY KEY,
    sa2_name TEXT,
    income_earners FLOAT,
    median_income FLOAT,
    mean_income FLOAT,
    gini_coefficient FLOAT
);
                  
DROP TABLE IF EXISTS postcode CASCADE;
CREATE TABLE postcode (
    postcode INTEGER,
    sa2_code BIGINT,
    sa2_name TEXT,
    PRIMARY KEY (postcode, sa2_code)
);
                  
DROP TABLE IF EXISTS stops CASCADE;
CREATE TABLE stops (
    stop_id TEXT,
    stop_name TEXT,
    stop_lat FLOAT,
    stop_lon FLOAT,
    sa2_code BIGINT,
    sa2_name TEXT,
    sa4_name TEXT,
    wheelchair_boarding INTEGER,
    geom geometry(Point, 4326)
);
                  
DROP TABLE IF EXISTS sa2_boundaries CASCADE;
CREATE TABLE sa2_boundaries (
    sa2_code21 TEXT PRIMARY KEY,
    sa2_name21 TEXT NOT NULL,
    sa4_code21 TEXT,
    sa4_name21 TEXT,
    areasqkm21 DOUBLE PRECISION,
    geom geometry(MultiPolygon, 4326)
);
                  
DROP TABLE IF EXISTS school_catchments CASCADE;
CREATE TABLE school_catchments (
    school_id TEXT PRIMARY KEY,
    catch_type TEXT NOT NULL,
    school_name TEXT NOT NULL,
    add_date DATE,
    geom geometry(MultiPolygon, 4326) NOT NULL
);
                  
                  

CREATE INDEX idx_catchments_geom ON school_catchments USING GIST (geom);
"""))
cleaned_population_df.to_sql(
    "population",
    con=conn,
    if_exists='append',
    index=False,
    method='multi',          
    chunksize=1000           
)
cleaned_businesses_df.to_sql(
    "businesses",
    con=conn,
    if_exists='append',
    index=False,
    method='multi',         
    chunksize=1000          
)
cleaned_income_df.to_sql(
    "income",
    con=conn,
    if_exists='append',
    index=False,
    method='multi',         
    chunksize=1000          
)
cleaned_postcode_df.to_sql(
    "postcodes",
    con=conn,
    if_exists='append',
    index=False,
    method='multi',         
    chunksize=1000          
)
cleaned_stops_df.to_sql(
    "stops",
    con=conn,
    if_exists='append',
    index=False,
    method='multi',         
    chunksize=1000          
)
cleaned_school_catchment.set_geometry("geom", inplace=True)
cleaned_sa2_gdf.to_postgis("sa2_boundaries", conn, if_exists="replace", index=False)
cleaned_school_catchment.to_postgis("school_catchments", conn, if_exists="replace", index=False)
conn.commit()


In [180]:
conn.close()